In [4]:
import requests, json
from bs4 import BeautifulSoup

In [ ]:
# scraping the job grouplists
rooturl = "https://www.nsp.cz/odborny-smer"
rootpage = requests.get(rooturl).text
rootsoup = BeautifulSoup(rootpage, 'html.parser')
sel = rootsoup.select(".bricklayer li a")
rootlinks = [a["href"] for a in sel]

In [ ]:
jobdict = {}

In [ ]:
for link in rootlinks:
    if link == "https://www.nsp.cz/odborny-podsmer-rozsireny/ozbrojene-sily-a-bezpecnostni-sbory":
        link = "https://www.nsp.cz/odborny-smer/ozbrojene-sily-a-bezpecnostni-sbory"
    print(link)
    group_json = requests.get(link + "?view=0&_format=json", headers={"x-requested-with": "XMLHttpRequest"}).json()
    for entry in group_json['data']:
        for subentry in entry['items']['s1']:
            if subentry['children']:
                for child in subentry['children']:
                    jobdict[child['title']] = {'url': child['urlSlug']}
            else:
                jobdict[subentry['title']] = {'url': subentry['urlSlug']}

In [ ]:
# scraping the individual pages
print(len(jobdict))
i = 0
for job in jobdict:
    i += 1
    print(i, end=" ")
    subsrc = requests.get("https://www.nsp.cz/jednotka-prace/" + jobdict[job]['url']).text
    subsrcsoup = BeautifulSoup(subsrc, 'html.parser')
    jobsrc = subsrcsoup.select(".search-results__results > li")
    jobdict[job]["skills"] = {}
    skills = {"odb_dov": "#TYPE_TECHNICAL_HARD_SKILL", "odb_zna": "#TYPE_SPECIFIC_HARD_SKILL", "obecne": "#TYPE_GENERIC_HARD_SKILL", "mekke": "#TYPE_SOFT_SKILL"}
    for skill in skills:
        subskills = {}
        skillsoup = subsrcsoup.select(skills[skill]+" tbody tr")

        for subskill in skillsoup:
            subskills[subskill.get_text().strip()] = len(subskill.select(".circle--green"))
        jobdict[job]["skills"][skill] = subskills

In [8]:
with open("../data/jobs.json",encoding="utf-8") as file:
    jobdict = json.loads(file.read())

for job in listjobdict:
    if not jobdict[job]["skills"]["mekke"] or not jobdict[job]["skills"]["obecne"]:
        
    else:
        for skill in list(jobdict[job]["skills"]["mekke"]):
            if skill == "":
                del(jobdict[job]["skills"]["mekke"][skill])
        for skill in list(jobdict[job]["skills"]["obecne"]):
            if skill == "":
                del(jobdict[job]["skills"]["obecne"][skill])

Dealer
Makléř
Specialista analýzy finančního trhu
Specialista bankovních strategií
Specialista devizových obchodů
Specialista kapitálových obchodů
Specialista metodik bankovních hotovostních operací
Specialista systému platebních karet
Disponent likvidace pojistných událostí
Disponent likvidace zahraničních pojistných událostí
Disponent vzniku pojištění
Disponent zajištění
Hlavní metodik v pojišťovnictví
Pojistný analytik
Pojistný matematik
Riskmanažer v pojišťovnictví
Vyšetřovatel v pojišťovnictví
Dealer asistent
Metodik bankovních hotovostních operací
Samostatný pracovník devizových obchodů
Samostatný pracovník kapitálových obchodů
Samostatný pracovník podnikatelských úvěrů
Samostatný pracovník řízení úvěrů
Samostatný pracovník správy úvěrového portfolia
Samostatný pracovník systému platebních karet
Samostatný pracovník vývoje bankovních produktů
Asistent riskmanažera v pojišťovnictví
Metodik v pojišťovnictví
Pojistný matematik asistent
Samostatný pojišťovací poradce
Samostatný praco

In [36]:
with open("../data/jobs.json","w",encoding="utf-8") as file:
    file.write(json.dumps(jobdict, ensure_ascii=False))